# Load and Prepare Raw data and Save tables


In [1]:
from typing import List
import requests
import json
import pandas as pd
import numpy as np
import time
import sys
from collections.abc import MutableMapping,Iterable
import datetime


def get_API_response(url):
    response_API = requests.get(url)
    print('Status Code-', response_API.status_code)
    print('Loading Data--')
    start = time.time()
    raw_data = response_API.text
    # Parse Json
    data = json.loads(raw_data)
    print('Time taken to load data-',round((time.time()-start),3))
    return data

In [4]:
R_ExtractDate = datetime.datetime.today().strftime("%Y-%m-%d")
frax_start_date= '2020-12-21'

## Basic info of stable coins (defi lama)

In [2]:
# List all stablecoins along with their circulating amounts from defilama
url_SC_basic_info = 'https://stablecoins.llama.fi/stablecoins?includePrices=true'
# ?includePrices=true
raw_SC_basic_info = get_API_response(url_SC_basic_info)
raw_SC_basic_info_list =raw_SC_basic_info['peggedAssets']
pdf_SC_basic_info = pd.json_normalize(raw_SC_basic_info_list, sep='_')


Status Code- 200
Loading Data--
Time taken to load data- 0.001


In [5]:
select_keys = ['id', 'name', 'symbol', 'gecko_id', 'pegType', 'priceSource', 'circulating_peggedUSD', 'chains', 'price']
pdf_SC_basic_info_2 = pdf_SC_basic_info[select_keys]
pdf_SC_basic_info_2['R_ExtractDate'] = R_ExtractDate

<ipython-input-5-8d727a883b5d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_SC_basic_info_2['R_ExtractDate'] = R_ExtractDate


In [8]:
# Save dataframe to csv file
# pdf_SC_basic_info_2.to_csv('../data_raw/StableCoins_basicInfo.csv',index=False)

In [9]:
pdf_SC_basic_info_2.head(7)

,id,name,symbol,gecko_id,pegType,priceSource,circulating_peggedUSD,chains,price,R_ExtractDate
0,1,Tether,USDT,tether,peggedUSD,chainlink,6.628422e+10,"[Tron, Ethereum, Solana, BSC, Omni, Avalanche,...",1.000131,2022-07-23
1,2,USD Coin,USDC,usd-coin,peggedUSD,chainlink,5.426444e+10,"[Ethereum, Solana, Tron, Avalanche, BSC, Polyg...",1.000094,2022-07-23
2,3,TerraClassicUSD,USTC,terrausd,peggedUSD,chainlink,1.127878e+10,"[Terra Classic, Ethereum, BSC, Avalanche, Poly...",0.037917,2022-07-23
3,4,Binance USD,BUSD,binance-usd,peggedUSD,chainlink,1.797362e+10,"[Ethereum, BSC, Kava, Astar, Harmony, Solana, ...",1.000000,2022-07-23
4,5,Dai,DAI,dai,peggedUSD,chainlink,6.891558e+09,"[Ethereum, Avalanche, BSC, Fantom, Polygon, Gn...",1.000000,2022-07-23
5,6,Frax,FRAX,frax,peggedUSD,chainlink,1.406059e+09,"[Ethereum, Fantom, Arbitrum, Moonriver, Polygo...",0.999787,2022-07-23
6,7,TrueUSD,TUSD,true-usd,peggedUSD,chainlink,1.224972e+09,"[Ethereum, Tron, BSC, Cronos, Polygon, Avalanc...",0.999696,2022-07-23


## Historical MCap of SCs (* starting date is changing)

In [10]:
# Get historical mcap sum of all stablecoins in all chain
stablecoin_id = 6
url_SC_Mcap_allChains = 'https://stablecoins.llama.fi/stablecoincharts/all?stablecoin='+str(stablecoin_id)
raw_SC_Mcap_allChainst = get_API_response(url_SC_Mcap_allChains)
raw_SC_Mcap_allChainst_list =raw_SC_Mcap_allChainst
pdf_SC_Mcap_allChainst= pd.json_normalize(raw_SC_Mcap_allChainst_list, sep='_')
pdf_SC_Mcap_allChainst['date'] = pd.to_datetime(pdf_SC_Mcap_allChainst['date'] , unit='s')
pdf_SC_Mcap_allChainst['R_ExtractDate'] = R_ExtractDate

Status Code- 200
Loading Data--
Time taken to load data- 0.041


In [11]:
pdf_SC_Mcap_allChainst

,date,totalCirculating_peggedUSD,totalUnreleased_peggedUSD,totalCirculatingUSD_peggedUSD,totalMintedUSD_peggedUSD,totalBridgedToUSD_peggedUSD,R_ExtractDate
0,2021-01-01,5.738905e+07,0,5.738905e+07,0,0,2022-07-23
1,2021-01-02,7.177263e+07,0,7.177263e+07,0,0,2022-07-23
2,2021-01-03,7.828972e+07,0,7.828972e+07,0,0,2022-07-23
3,2021-01-04,7.887698e+07,0,7.887698e+07,0,0,2022-07-23
4,2021-01-05,7.627737e+07,0,7.627737e+07,0,0,2022-07-23
...,...,...,...,...,...,...,...
564,2022-07-19,1.366059e+09,0,1.366434e+09,0,0,2022-07-23
565,2022-07-20,1.386059e+09,0,1.386439e+09,0,0,2022-07-23
566,2022-07-21,1.406059e+09,0,1.405845e+09,0,0,2022-07-23
567,2022-07-22,1.406059e+09,0,1.405845e+09,0,0,2022-07-23


In [54]:
pdf_SC_basic_info_2['id'][i]

'39'

In [12]:
list_pdf_SC_Mcap_allChainst =[]
for i in range(pdf_SC_basic_info_2.shape[0]):
    try:
        print(i,pdf_SC_basic_info_2['name'][i], pdf_SC_basic_info_2['symbol'][i])
        
        stablecoin_id = pdf_SC_basic_info_2['id'][i]
        symbol = pdf_SC_basic_info_2['symbol'][i]

        url_SC_Mcap_allChains = 'https://stablecoins.llama.fi/stablecoincharts/all?stablecoin='+str(stablecoin_id)
        raw_SC_Mcap_allChainst = get_API_response(url_SC_Mcap_allChains)
        
        pdf_SC_Mcap_allChainst= pd.json_normalize(raw_SC_Mcap_allChainst, sep='_')
        pdf_SC_Mcap_allChainst['date'] = pd.to_datetime(pdf_SC_Mcap_allChainst['date'] , unit='s')
        pdf_SC_Mcap_allChainst = pdf_SC_Mcap_allChainst[pdf_SC_Mcap_allChainst['date']>= frax_start_date]
        pdf_SC_Mcap_allChainst['id'] = stablecoin_id
        pdf_SC_Mcap_allChainst['symbol'] = symbol
        pdf_SC_Mcap_allChainst = pdf_SC_Mcap_allChainst[['id','symbol','date','totalCirculating_peggedUSD','totalUnreleased_peggedUSD','totalCirculatingUSD_peggedUSD','totalMintedUSD_peggedUSD','totalBridgedToUSD_peggedUSD']]
        list_pdf_SC_Mcap_allChainst.append(pdf_SC_Mcap_allChainst)
    except Exception as e:
        print('Not present and skipping..')
pdf_SC_Mcap_allChains = pd.DataFrame()
pdf_SC_Mcap_allChains = pd.concat(list_pdf_SC_Mcap_allChainst)
pdf_SC_Mcap_allChains.reset_index(inplace=True)
pdf_SC_Mcap_allChains.drop('index',axis=1, inplace = True)
pdf_SC_Mcap_allChains['R_ExtractDate'] = R_ExtractDate


0 Tether USDT
Status Code- 200
Loading Data--
Time taken to load data- 0.008
1 USD Coin USDC
Status Code- 200
Loading Data--
Time taken to load data- 0.0
2 TerraClassicUSD USTC
Status Code- 200
Loading Data--
Time taken to load data- 0.0
3 Binance USD BUSD
Status Code- 200
Loading Data--
Time taken to load data- 0.0
4 Dai DAI
Status Code- 200
Loading Data--
Time taken to load data- 0.0
5 Frax FRAX
Status Code- 200
Loading Data--
Time taken to load data- 0.01
6 TrueUSD TUSD
Status Code- 200
Loading Data--
Time taken to load data- 0.008
7 Liquity USD LUSD
Status Code- 200
Loading Data--
Time taken to load data- 0.008
8 Fei USD FEI
Status Code- 200
Loading Data--
Time taken to load data- 0.0
9 Magic Internet Money MIM
Status Code- 200
Loading Data--
Time taken to load data- 0.0
10 Pax Dollar USDP
Status Code- 200
Loading Data--
Time taken to load data- 0.0
11 Neutrino USD USDN
Status Code- 200
Loading Data--
Time taken to load data- 0.008
12 YUSD Stablecoin YUSD
Status Code- 200
Loading D

In [16]:
# Starting date is now 2021-01-01 (ok from Frax point of view)
pdf_SC_Mcap_allChains

,id,symbol,date,totalCirculating_peggedUSD,totalUnreleased_peggedUSD,totalCirculatingUSD_peggedUSD,totalMintedUSD_peggedUSD,totalBridgedToUSD_peggedUSD,R_ExtractDate
0,1,USDT,2021-01-01,2.093403e+10,0.0,2.093403e+10,0,0,2022-07-23
1,1,USDT,2021-01-02,2.112364e+10,0.0,2.112364e+10,0,0,2022-07-23
2,1,USDT,2021-01-03,2.120298e+10,0.0,2.120298e+10,0,0,2022-07-23
3,1,USDT,2021-01-04,2.135052e+10,0.0,2.135052e+10,0,0,2022-07-23
4,1,USDT,2021-01-05,2.180823e+10,0.0,2.180823e+10,0,0,2022-07-23
...,...,...,...,...,...,...,...,...,...
14173,58,3USD,2022-07-23,1.571778e+06,0.0,1.571778e+06,0,0,2022-07-23
14174,59,SIGUSD,2022-07-20,7.893999e+05,0.0,7.893999e+05,0,0,2022-07-23
14175,59,SIGUSD,2022-07-21,7.893999e+05,0.0,7.893999e+05,0,0,2022-07-23
14176,59,SIGUSD,2022-07-22,7.918877e+05,0.0,7.918877e+05,0,0,2022-07-23


In [17]:
# Save dataframe to csv file
# pdf_SC_Mcap_allChains.to_csv('../data_raw/StableCoins_hist_mcap_allChains.csv',index=False)

# Historical price of SC

In [18]:
# Get historical price of all stable coins 
# Price data is not useful as starting date is 2022-05-10
url_SC_price = 'https://stablecoins.llama.fi/stablecoinprices'
raw_SC_price = get_API_response(url_SC_price) 
raw_SC_price_list = raw_SC_price
pdf_SC_hist_price = pd.json_normalize(raw_SC_price_list, sep='_')
pdf_SC_hist_price['date'] = pd.to_datetime(pdf_SC_hist_price['date'] , unit='s') 
new_columns = [col_str.replace('prices_','') for col_str in pdf_SC_hist_price.columns]
pdf_SC_hist_price.columns = new_columns
pdf_SC_hist_price['R_ExtractDate'] = R_ExtractDate

Status Code- 200
Loading Data--
Time taken to load data- 0.014


In [37]:
pdf_SC_hist_price.head(5)

,date,paxos-standard,usdx,tether,binance-usd,origin-dollar,gemini-dollar,musd,nusd,terrausd,...,celo-euro,seur,stasis-eurs,euro-coin,par-stablecoin,ageur,hedge-usd,sigmausd,three-usd,R_ExtractDate
0,2021-01-23,0.998736,0.818470,1.000412,1.000542,0.986583,1.010310,1.003928,1.008048,1.007768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-23
1,2021-01-24,1.000907,0.821758,0.998582,1.000039,1.002471,1.008093,0.997771,1.010669,1.010426,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-23
2,2021-01-25,0.999918,0.809376,0.999586,0.999274,1.009678,1.013200,1.013282,1.008034,1.000588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-23
3,2021-01-26,1.000255,0.810791,1.000846,1.001361,1.001880,1.019425,0.997337,1.006399,0.999488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-23
4,2021-01-27,0.998193,0.803431,1.003201,0.999814,1.002282,1.016535,1.009880,1.009040,1.012201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-23


In [38]:
# Save dataframe to csv file
# pdf_SC_hist_price.to_csv('../data_raw/StableCoins_hist_price.csv',index=False)

## TVL related data (defilama)


In [40]:
# Get All protocols in defilama along with their tvl
url_tvl_protocols = 'https://api.llama.fi/protocols'
raw_tvl_protocols = get_API_response(url_tvl_protocols) 
raw_tvl_protocols_list = raw_tvl_protocols
pdf_tvl_protocols_allcols = pd.json_normalize(raw_tvl_protocols_list, sep='_')


Status Code- 200
Loading Data--
Time taken to load data- 0.024


In [41]:
# Select important coplumns
select_columns = ['id', 'name', 'address', 'symbol','chain', 'gecko_id',\
    'audits', 'cmcId','category', 'chains' ,'oracles', 'slug','tvl','fdv', 'mcap']
    
pdf_tvl_protocols = pdf_tvl_protocols_allcols[select_columns]
# add extract date as the values are based in current date
pdf_tvl_protocols['R_ExtractDate'] = R_ExtractDate
# 1738 protocols enlisted

# All columns
# ['id', 'name', 'address', 'symbol', 'url', 
# 'description', 'chain', 'logo', 'audits', 
# 'audit_note', 'gecko_id', 'cmcId', 'category', 
# 'chains', 'module', 'twitter', 'audit_links', 
# 'oracles', 'slug', 'tvl', 'chainTvls', 
# 'change_1h', 'change_1d', 'change_7d', 'fdv', 'mcap']

# Extract top 50 tvl protocols on extract date
pdf_protocols_tvl_top50 = pdf_tvl_protocols.sort_values(by='tvl',ascending=False).head(50)
# Extract top 50 tvl protocols on extract date
pdf_protocols_mcap_top50 = pdf_tvl_protocols.sort_values(by='mcap',ascending=False).head(50)

<ipython-input-41-5a8f3737fbea>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_tvl_protocols['R_ExtractDate'] = R_ExtractDate


In [30]:
pdf_protocols_tvl_top50.head(5)

,id,name,address,symbol,chain,gecko_id,audits,cmcId,category,chains,oracles,slug,tvl,fdv,mcap,R_ExtractDate
0,118,MakerDAO,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,MKR,Ethereum,maker,2,1518,CDP,[Ethereum],[Maker],makerdao,8.565404e+09,1.054898e+09,9.455181e+08,2022-07-20
1,240,Polygon Bridge & Staking,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,MATIC,Polygon,matic-network,0,3890,Chain,[Polygon],NaN,polygon-bridge-&-staking,7.490970e+09,9.418557e+09,7.541254e+09,2022-07-20
2,182,Lido,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,Multi-Chain,lido-dao,2,8000,Liquid Staking,"[Ethereum, Solana, Moonriver, Terra]",NaN,lido,6.705730e+09,1.640187e+09,8.197632e+08,2022-07-20
3,1,Uniswap,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,Multi-Chain,uniswap,2,7083,Dexes,"[Ethereum, Polygon, Arbitrum, Optimism, Celo]",[],uniswap,5.949621e+09,7.506685e+09,3.426724e+09,2022-07-20
4,3,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,Multi-Chain,curve-dao-token,2,6538,Dexes,"[Ethereum, Polygon, Avalanche, Fantom, Arbitru...",[],curve,5.910075e+09,4.159745e+09,4.936212e+08,2022-07-20


In [31]:
pdf_protocols_mcap_top50.head()

,id,name,address,symbol,chain,gecko_id,audits,cmcId,category,chains,oracles,slug,tvl,fdv,mcap,R_ExtractDate
1,240,Polygon Bridge & Staking,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,MATIC,Polygon,matic-network,0,3890,Chain,[Polygon],NaN,polygon-bridge-&-staking,7.490970e+09,9.418557e+09,7.541254e+09,2022-07-20
5,2,WBTC,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,Ethereum,wrapped-bitcoin,2,3717,Bridge,[Ethereum],[],wbtc,5.557608e+09,5.571891e+09,5.571891e+09,2022-07-20
244,221,Defi Swap,0xa0b73e1ff0b80914ab6fe0444e65848c4c34450b,CRO,Ethereum,crypto-com-chain,2,1776,Dexes,[Ethereum],NaN,defi-swap,2.262947e+07,NaN,3.466117e+09,2022-07-20
3,1,Uniswap,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,Multi-Chain,uniswap,2,7083,Dexes,"[Ethereum, Polygon, Arbitrum, Optimism, Celo]",[],uniswap,5.949621e+09,7.506685e+09,3.426724e+09,2022-07-20
1557,1065,The Sandbox,0x3845badAde8e6dFF049820680d1F14bD3903a5d0,SAND,Multi-Chain,the-sandbox,0,6210,Gaming,"[Ethereum, Polygon]",NaN,the-sandbox,0.000000e+00,4.462094e+09,1.880462e+09,2022-07-20


In [42]:
# Save dataframe to csv file
# pdf_protocols_tvl_top50.to_csv('../data_raw/Protocols_tvl_top50.csv',index=False)
# pdf_protocols_mcap_top50.to_csv('../data_raw/Protocols_mcap_top50.csv',index=False)

In [67]:
# pdf_tvl_protocols['category'].unique()
# ['CDP', 'Chain', 'Liquid Staking', 'Dexes', 'Bridge', 'Lending',
#        'Yield', 'Services', 'Algo-Stables', 'Yield Aggregator',
#        'Derivatives', 'Cross Chain', 'Privacy', 'Reserve Currency',
#        'Insurance', 'Payments', 'Launchpad', 'Synthetics', 'Options',
#        'Indexes', 'NFT Lending', 'RWA', 'Staking', 'Farm',
#        'NFT Marketplace', 'Gaming', 'Prediction Market', 'Oracle']

array(['CDP', 'Chain', 'Liquid Staking', 'Dexes', 'Bridge', 'Lending',
       'Yield', 'Services', 'Algo-Stables', 'Yield Aggregator',
       'Derivatives', 'Cross Chain', 'Privacy', 'Reserve Currency',
       'Insurance', 'Payments', 'Launchpad', 'Synthetics', 'Options',
       'Indexes', 'NFT Lending', 'RWA', 'Staking', 'Farm',
       'NFT Marketplace', 'Gaming', 'Prediction Market', 'Oracle'],
      dtype=object)

In [7]:
# Select important coplumns
select_columns = ['id', 'name', 'symbol','url', 'description', 'gecko_id',\
    'cmcId','category', 'slug','tvl','fdv', 'mcap']
    
pdf_tvl_protocols_test = pdf_tvl_protocols_allcols[select_columns]


In [44]:
pdf_protocols_tvlMcap_DLCategory_AlgoStables = pdf_tvl_protocols[pdf_tvl_protocols['category']=='Algo-Stables']
pdf_protocols_tvlMcap_DLCategory_ReserveCurrency = pdf_tvl_protocols[pdf_tvl_protocols['category']=='Reserve Currency']
pdf_protocols_tvlMcap_DLCategory_CDP = pdf_tvl_protocols[pdf_tvl_protocols['category']=='CDP']

In [47]:
pdf_protocols_tvlMcap_DLCategory_AlgoStables.to_csv('../data_raw/Protocols_tvlMcap_DLCategory_AlgoStables.csv',index=False)
pdf_protocols_tvlMcap_DLCategory_ReserveCurrency.to_csv('../data_raw/Protocols_tvlMcap_DLCategory_ReserveCurrency.csv',index=False)
pdf_protocols_tvlMcap_DLCategory_CDP.to_csv('../data_raw/Protocols_tvlMcap_DLCategory_CDP.csv',index=False)